# ROCCO Quick Start Demo

## BAM Preprocessing

**Download Input Alignments:** To download and index the ATAC-seq alignments (human lymphoblast) used for this demo, run
```
xargs -L 1 curl -O -J -L < demo_bams.txt
samtools index -M *.bam
```
in the main `ROCCO` directory.

These files are are obtained from the ENCODE project with the follwing [query](https://www.encodeproject.org/search/?type=Experiment&control_type%21=%2A&status=released&perturbed=false&assay_title=ATAC-seq&biosample_ontology.cell_slims=lymphoblast&audit.ERROR.category%21=extremely+low+read+depth&audit.NOT_COMPLIANT.category%21=low+FRiP+score&audit.NOT_COMPLIANT.category%21=poor+library+complexity&audit.NOT_COMPLIANT.category%21=severe+bottlenecking&audit.WARNING.category%21=moderate+library+complexity&audit.WARNING.category%21=mild+to+moderate+bottlenecking&audit.WARNING.category%21=moderate+number+of+reproducible+peaks).

The downloaded alignment files have been QC-processed with the [ENCODE ATAC-seq pipeline](https://www.encodeproject.org/atac-seq/). In general, we assume the BAM files used as input to ROCCO have been prepared according to some QC standard---duplicate removal, adapter trimming, etc.

## `rocco prep`

This step generates a smooth signal track for each sample's BAM file and then divides each into chromosome-specific directories `tracks_<chromosome name>`, thereby providing ROCCO conformable input.

See `rocco prep -h` for additional details. 

In [1]:
!rocco prep --bamdir .

(CVXPY) Aug 22 11:43:54 PM: Encountered unexpected exception importing solver GLPK:
ImportError("cannot import name 'glpk' from 'cvxopt' (/nas/longleaf/rhel8/apps/python/3.9.6/lib/python3.9/site-packages/cvxopt-1.2.6-py3.9-linux-x86_64.egg/cvxopt/__init__.py)")
(CVXPY) Aug 22 11:43:54 PM: Encountered unexpected exception importing solver GLPK_MI:
ImportError("cannot import name 'glpk' from 'cvxopt' (/nas/longleaf/rhel8/apps/python/3.9.6/lib/python3.9/site-packages/cvxopt-1.2.6-py3.9-linux-x86_64.egg/cvxopt/__init__.py)")
/work/users/n/h/nolanh/quickstart/ROCCO/ENCFF009NCL.bam: running bamSitesToWig.py
/work/users/n/h/nolanh/quickstart/ROCCO/ENCFF110EWQ.bam: running bamSitesToWig.py
/work/users/n/h/nolanh/quickstart/ROCCO/ENCFF231YYD.bam: running bamSitesToWig.py
/work/users/n/h/nolanh/quickstart/ROCCO/ENCFF395ZMS.bam: running bamSitesToWig.py
/work/users/n/h/nolanh/quickstart/ROCCO/ENCFF495DQP.bam: running bamSitesToWig.py
/work/users/n/h/nolanh/quickstart/ROCCO/ENCFF621AYF.bam: runnin

## `rocco chrom`

Run on chromosome 21 with default parameters

In [2]:
!rocco chrom --wig_path tracks_chr21 --chrom chr21

ROCCO_chrom: reading wig file tracks_chr21/chr21_ENCFF009NCL.bam.bw.wig
ROCCO_chrom: reading wig file tracks_chr21/chr21_ENCFF110EWQ.bam.bw.wig
ROCCO_chrom: reading wig file tracks_chr21/chr21_ENCFF231YYD.bam.bw.wig
ROCCO_chrom: reading wig file tracks_chr21/chr21_ENCFF395ZMS.bam.bw.wig
ROCCO_chrom: reading wig file tracks_chr21/chr21_ENCFF495DQP.bam.bw.wig
ROCCO_chrom: reading wig file tracks_chr21/chr21_ENCFF621AYF.bam.bw.wig
ROCCO_chrom: reading wig file tracks_chr21/chr21_ENCFF767FGV.bam.bw.wig
ROCCO_chrom: reading wig file tracks_chr21/chr21_ENCFF797EAL.bam.bw.wig
ROCCO_chrom: reading wig file tracks_chr21/chr21_ENCFF801THG.bam.bw.wig
ROCCO_chrom: reading wig file tracks_chr21/chr21_ENCFF948HNW.bam.bw.wig
ROCCO_chrom: writing output: ./ROCCO_out_chr21_0.035_1.0_0.0_1.0_1.0_1.0.bed
